In [35]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [36]:
# Drop the non-beneficial ID columns, 'EIN'	.
df = application_df.drop(columns=['EIN'])

In [37]:
# Determine the number of unique values in each column.
df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [38]:
# Look at APPLICATION_TYPE value counts for binning
application_types = df['APPLICATION_TYPE'].value_counts()
application_types

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [39]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = [index for index,row in application_types.items() if row < 500]


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [40]:
# Look at CLASSIFICATION value counts for binning
classification_types = df['CLASSIFICATION'].value_counts()
classification_types

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [41]:
# You may find it helpful to look at CLASSIFICATION value counts >1
[[index,row] for index,row in classification_types.items() if row > 1]

[['C1000', 17326],
 ['C2000', 6074],
 ['C1200', 4837],
 ['C3000', 1918],
 ['C2100', 1883],
 ['C7000', 777],
 ['C1700', 287],
 ['C4000', 194],
 ['C5000', 116],
 ['C1270', 114],
 ['C2700', 104],
 ['C2800', 95],
 ['C7100', 75],
 ['C1300', 58],
 ['C1280', 50],
 ['C1230', 36],
 ['C1400', 34],
 ['C7200', 32],
 ['C2300', 32],
 ['C1240', 30],
 ['C8000', 20],
 ['C7120', 18],
 ['C1500', 16],
 ['C1800', 15],
 ['C6000', 15],
 ['C1250', 14],
 ['C8200', 11],
 ['C1238', 10],
 ['C1278', 10],
 ['C1235', 9],
 ['C1237', 9],
 ['C7210', 7],
 ['C2400', 6],
 ['C1720', 6],
 ['C4100', 6],
 ['C1257', 5],
 ['C1600', 5],
 ['C1260', 3],
 ['C2710', 3],
 ['C0', 3],
 ['C3200', 2],
 ['C1234', 2],
 ['C1246', 2],
 ['C1267', 2],
 ['C1256', 2]]

In [42]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = [index for index,row in classification_types.items() if row < classification_types.mean()]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [44]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(df[['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
        'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']])
processed_df = pd.concat([df[['STATUS', 'ASK_AMT']], dummies], axis=1)

In [45]:
# Split our preprocessed data into our features and target arrays
X = processed_df
y = df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.2, random_state=42)

In [46]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [47]:
model = tf.keras.models.Sequential()



# First hidden layer
model.add(tf.keras.layers.Dense(64, input_shape=(112,), activation='relu'))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=3, activation='tanh'))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
model.build()
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                7232      
                                                                 
 dense_10 (Dense)            (None, 3)                 195       
                                                                 
 dense_11 (Dense)            (None, 1)                 4         
                                                                 
Total params: 7431 (29.03 KB)
Trainable params: 7431 (29.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [48]:
# Compile the model

model.compile(loss="BinaryCrossentropy", optimizer="adam", metrics=['accuracy', 'mse'])

In [49]:
# Train the model
fit_model = model.fit(X_train_scaled, y_train, epochs=75)

Epoch 1/75
858/858 [==============================] - 3s 2ms/step - loss: 0.5778 - accuracy: 0.7180 - mse: 0.1958
Epoch 2/75
858/858 [==============================] - 3s 3ms/step - loss: 0.5620 - accuracy: 0.7256 - mse: 0.1894
Epoch 3/75
858/858 [==============================] - 2s 2ms/step - loss: 0.5595 - accuracy: 0.7262 - mse: 0.1885
Epoch 4/75
858/858 [==============================] - 2s 2ms/step - loss: 0.5582 - accuracy: 0.7275 - mse: 0.1880
Epoch 5/75
858/858 [==============================] - 2s 2ms/step - loss: 0.5570 - accuracy: 0.7285 - mse: 0.1877
Epoch 6/75
858/858 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7282 - mse: 0.1875
Epoch 7/75
858/858 [==============================] - 2s 2ms/step - loss: 0.5565 - accuracy: 0.7274 - mse: 0.1876
Epoch 8/75
858/858 [==============================] - 3s 3ms/step - loss: 0.5556 - accuracy: 0.7274 - mse: 0.1872
Epoch 9/75
858/858 [==============================] - 2s 3ms/step - loss: 0.5552 - accur

In [52]:
# Evaluate the model using the test data
model_loss = model.evaluate(X_test_scaled, y_test, verbose=2)[0]
model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)[1]

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5624 - accuracy: 0.7242 - mse: 0.1898 - 283ms/epoch - 1ms/step
215/215 - 0s - loss: 0.5624 - accuracy: 0.7242 - mse: 0.1898 - 290ms/epoch - 1ms/step
Loss: 0.5623909831047058, Accuracy: 0.7241982221603394


In [53]:
# Export our model to HDF5 file
model.save("AlphabetSoupCharity_Optimization.h5")